In [279]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [280]:
merged_df = pd.read_excel('data_interpolated.xlsx')
merged_df_encoded = merged_df
print("열 이름 목록:", merged_df.columns.tolist())
# merged_df_encoded = pd.get_dummies(merged_df, columns=['지역'], drop_first=False)
# region_cols = [col for col in merged_df_encoded.columns if col.startswith('지역_')]
# cities = [col.replace('지역_', '') for col in region_cols]
# print("분석할 도시 목록 (drop_first=False):", cities)

# 1.3. 타겟 변수 분리
if '시군구 합계 출산율 2015-2023' in merged_df_encoded.columns:
    merged_df_encoded = merged_df_encoded.rename(columns={'시군구 합계 출산율 2015-2023': '출산율'})
else:
    print("'시군구 합계 출산율 2015-2023' 열이 데이터에 존재하지 않습니다. 타겟 변수를 확인하세요.")

if '출산율' in merged_df_encoded.columns:
    X = merged_df_encoded.drop(['출산율', '연도'], axis=1)
    y = merged_df_encoded['출산율']
else:
    print("'출산율' 열이 데이터에 존재하지 않습니다. 타겟 변수를 확인하세요.")


열 이름 목록: ['지역', '연도', '1인당 GRDP 2015-2022', '근로시간_상용근로일수 (일)', '근로시간_상용소정실근로시간 (시간)', '근로시간_상용월급여액 (원)', '근로시간_상용정액급여 (원)', '근로시간_상용초과근로시간 (시간)', '근로시간_상용초과급여 (원)', '근로시간_상용총근로시간 (시간)', '근로시간_상용특별급여 (원)', '시군구 합계 출산율 2015-2023', '지역별 1인가구 비율', '지역별 건강생활 실천율 2015-2023', '지역별 경제 참여율 2015-2023', '지역별 고용률 2015-2023', '지역별 고위험 음주율 2015-2023', '지역별 노령화 지수 215-2023', '지역별 문화기반시설 수 (인구 10만명 당) 2015-2022', '지역별 문화생활 인구 비율 2015-2023(2년 단위)', '지역별 사교육비', '지역별 성평등 지수 2017-2022', '지역별 소득 2015-2022', '지역별 스트레스 인지율 2015-2023', '지역별 실업률 2015-2023', '지역별 여성 경제 참여율 2015-2023', '지역별 여성 고용률 2015-2023', '지역별 여성 실업률 2015-2023', '지역별 여성 청년층 (19-39) 비율 2015-2023', '지역별 여성 초혼 연령 2015-2023', '지역별 월평균 임금 2015-2023', '지역별 음주율 2015-2023', '지역별 인구밀도 2015-2023', '지역별 조이혼율 2015-2023', '지역별 조혼인율', '지역별 청년층(19-39) 비율 2015-2023', '지역별 평균 연령 2015-2023']


In [281]:

# 통계적 요약 확인
print(merged_df_encoded.describe())
print("결측치 개수:\n", merged_df_encoded.isnull().sum())

# 결측치 비율 확인
print("결측치 비율:\n", merged_df_encoded.isnull().mean())

                연도  1인당 GRDP 2015-2022  근로시간_상용근로일수 (일)  근로시간_상용소정실근로시간 (시간)  \
count   216.000000          216.000000       216.000000           216.000000   
mean   2019.000000        39823.101852        21.131019           163.810648   
std       2.587987        11046.670386         0.773161             6.441777   
min    2015.000000        22685.000000        19.000000           149.000000   
25%    2017.000000        32267.000000        20.600000           159.150000   
50%    2019.000000        36350.000000        21.100000           163.000000   
75%    2021.000000        46002.000000        21.600000           168.600000   
max    2023.000000        76234.000000        23.100000           179.400000   

       근로시간_상용월급여액 (원)  근로시간_상용정액급여 (원)  근로시간_상용초과근로시간 (시간)  근로시간_상용초과급여 (원)  \
count     2.160000e+02     2.160000e+02          216.000000       216.000000   
mean      3.117062e+06     2.847914e+06           12.781019    269147.240741   
std       3.394953e+05     3.258983e+05

In [282]:
df = merged_df_encoded.copy()
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler_y = StandardScaler()

# Reshape y for scaling
y_scaled = scaler_y.fit_transform(df[['출산율']])



# Update the DataFrame
df['출산율_Scaled'] = y_scaled

In [283]:

# 타겟 변수와 연도 제거
y = df['출산율_Scaled']
X = df.drop(['출산율', '출산율_Scaled', '연도','지역'], axis=1)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 데이터프레임으로 변환
X_train_df = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_df = pd.DataFrame(X_test_scaled, columns=X.columns)

# VIF 계산 함수
def calculate_vif(X):
    vif = pd.DataFrame()
    vif['Feature'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif

# VIF 기반 변수 제거
def iterative_vif_removal(X, threshold=30):
    while True:
        vif_df = calculate_vif(X)
        high_vif_features = vif_df[vif_df['VIF'] > threshold]['Feature']
        if high_vif_features.empty:
            break
        # VIF가 가장 높은 변수 제거
        X = X.drop(columns=high_vif_features.iloc[0])
    return X, vif_df

# VIF 감소 실행
X_reduced, final_vif_df = iterative_vif_removal(X_train_df, threshold=10)

print("최종 VIF 감소 후 특성:\n", X_reduced.columns)
print("최종 VIF 값:\n", final_vif_df)

# 테스트 데이터셋에도 동일한 컬럼 적용
X_test_reduced = X_test_df[X_reduced.columns]


최종 VIF 감소 후 특성:
 Index(['근로시간_상용총근로시간 (시간)', '근로시간_상용특별급여 (원)', '지역별 건강생활 실천율 2015-2023',
       '지역별 고위험 음주율 2015-2023', '지역별 문화생활 인구 비율 2015-2023(2년 단위)', '지역별 사교육비',
       '지역별 성평등 지수 2017-2022', '지역별 스트레스 인지율 2015-2023',
       '지역별 여성 고용률 2015-2023', '지역별 여성 실업률 2015-2023',
       '지역별 여성 초혼 연령 2015-2023', '지역별 월평균 임금 2015-2023', '지역별 음주율 2015-2023',
       '지역별 인구밀도 2015-2023', '지역별 조이혼율 2015-2023', '지역별 평균 연령 2015-2023'],
      dtype='object')
최종 VIF 값:
                             Feature       VIF
0                 근로시간_상용총근로시간 (시간)  1.873862
1                   근로시간_상용특별급여 (원)  2.487824
2            지역별 건강생활 실천율 2015-2023  4.511088
3             지역별 고위험 음주율 2015-2023  5.881748
4   지역별 문화생활 인구 비율 2015-2023(2년 단위)  3.714748
5                          지역별 사교육비  2.112708
6              지역별 성평등 지수 2017-2022  3.181215
7            지역별 스트레스 인지율 2015-2023  2.041834
8              지역별 여성 고용률 2015-2023  3.198808
9              지역별 여성 실업률 2015-2023  2.361631
10           지역별 여성 초혼 연령 2

In [284]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# Lasso 모델 정의
lasso = Lasso(alpha=0.005)  # alpha 값은 교차 검증을 통해 조정 가능

# 모델 학습
lasso.fit(X_reduced, y_train)

# 변수 선택
model = SelectFromModel(lasso, prefit=True)
X_lasso = model.transform(X_reduced)

# 선택된 변수 이름
selected_features = X_reduced.columns[model.get_support()]
print("선택된 변수 목록:", selected_features.tolist())

# 테스트 데이터에 동일한 변수 적용
X_test_lasso = X_test_reduced[selected_features]


선택된 변수 목록: ['근로시간_상용총근로시간 (시간)', '근로시간_상용특별급여 (원)', '지역별 건강생활 실천율 2015-2023', '지역별 사교육비', '지역별 성평등 지수 2017-2022', '지역별 스트레스 인지율 2015-2023', '지역별 여성 고용률 2015-2023', '지역별 여성 실업률 2015-2023', '지역별 여성 초혼 연령 2015-2023', '지역별 월평균 임금 2015-2023', '지역별 음주율 2015-2023', '지역별 인구밀도 2015-2023', '지역별 조이혼율 2015-2023', '지역별 평균 연령 2015-2023']


c:\Users\yooji\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [285]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

# 2. 트리 기반 모델 적용 (XGBoost)
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,          # 트리 수를 줄여 과적합 방지
    learning_rate=0.005,        # 낮은 학습률로 안정적인 학습
    max_depth=3,               # 낮은 트리 깊이로 모델 단순화
    subsample=0.7,             # 데이터 샘플링 비율 감소
    colsample_bytree=1.0,      # 특성 샘플링 비율 감소
    gamma=1,                   # 분할 최소 손실 감소
    min_child_weight=2,        # 자식 노드의 최소 가중치

    random_state=42,
    verbosity=0
)
# 모델 학습
xgb_model.fit(
    X_train[selected_features], y_train,
    eval_set=[(X_test[selected_features], y_test)],
    verbose=False
)




XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.005, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [287]:
 #예측 및 평가
y_pred_xgb = xgb_model.predict(X_test[selected_features])
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)
print(f'XGBoost MSE: {mse_xgb:.2f}, R²: {r2_xgb:.2f}')



NotFittedError: need to call fit or load_model beforehand

In [288]:
import lightgbm as lgb

# LightGBM 모델 정의
lgb_model = lgb.LGBMRegressor(
    objective='regression',
    n_estimators=100,          # Reduced number of trees
    learning_rate=0.05,        # Lower learning rate
    max_depth=3,               # Shallower trees
    min_child_samples=5,       # Minimum samples per leaf
    num_leaves=31,             # Controls tree complexity
    subsample=0.7,             # Subsample ratio for training instances
    colsample_bytree=0.7,      # Subsample ratio for features
    gamma=1,                   # Minimum loss reduction to make a split
    min_child_weight=2,        # Minimum sum of instance weight needed in a child
    reg_alpha=0.1,             # L1 regularization
    reg_lambda=1,              # L2 regularization
    random_state=42,
    verbosity=-1
)

# Initialize Leave-One-Out Cross-Validation

lgb_model.fit(
    X_train[selected_features], y_train,
    eval_set=[(X_test[selected_features], y_test)],
)

LGBMRegressor(colsample_bytree=0.7, gamma=1, learning_rate=0.05, max_depth=3,
              min_child_samples=5, min_child_weight=2, objective='regression',
              random_state=42, reg_alpha=0.1, reg_lambda=1, subsample=0.7,
              verbosity=-1)

In [289]:
from sklearn.model_selection import LeaveOneOut, cross_val_score

# 예측 및 평가
y_pred_lgb = lgb_model.predict(X_test[selected_features])
mse_lgb = mean_squared_error(y_test, y_pred_lgb)
r2_lgb = r2_score(y_test, y_pred_lgb)
print(f'LightGBM MSE: {mse_lgb:.2f}, R²: {r2_lgb:.2f}')



LightGBM MSE: 0.04, R²: 0.96


In [291]:

# 3. SHAP 분석
for model in [xgb_model,lgb_model]:
    explainer = shap.Explainer(model)
    shap_values = explainer(X_test[selected_features])
    from matplotlib import rc

    # 한글 폰트 설정
    plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows의 경우 "맑은 고딕"
    # plt.rcParams['font.family'] = 'AppleGothic' # macOS의 경우
    # plt.rcParams['font.family'] = 'NanumGothic' # Ubuntu의 경우

    # 음수 기호 깨짐 방지
    plt.rcParams['axes.unicode_minus'] = False
    # 변수 중요도 시각화 (막대 그래프)
    shap.summary_plot(shap_values, X_test[selected_features], plot_type="bar")


TypeError: The passed model is not callable and cannot be analyzed directly with the given masker! Model: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [ ]:
plt.title("Variable Importance (SHAP)")
plt.show()

# 변수 중요도 시각화 (점 플롯)
shap.summary_plot(shap_values, X_test[selected_features])
plt.title("SHAP Summary Plot")
plt.show()